<a href="https://colab.research.google.com/github/sigeek/Visual-Relationship-Detection-LTN/blob/master/predicate_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installation, create files

In [1]:
!pip install anytree
!pip install rdflib
!pip install tensorflow==1.15.5

     |████████████████████████████████| 51kB 5.6MB/s 
     |████████████████████████████████| 235kB 22.3MB/s 
     |████████████████████████████████| 51kB 4.9MB/s 
     |████████████████████████████████| 110.5MB 87kB/s 
     |████████████████████████████████| 51kB 6.6MB/s 
     |████████████████████████████████| 20.1MB 502kB/s 
     |████████████████████████████████| 512kB 47.5MB/s 
     |████████████████████████████████| 3.8MB 47.0MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp37-none-any.whl size=7540 sha256=ad31f420ebce9dfcb1cb41692db29bec395fffd324a897c1f0f0171c6a86bb35
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow-probability 0.12.1 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.

In [ ]:
import json
import csv
input_file = "data/sg_dataset/annotations_train.json"
output_file = 'data/sg_dataset/predicates_train.csv'

f = open(input_file,)
data = json.load(f)
with open(output_file, 'w', newline='') as file:
  writer = csv.writer(file) 
  for key in data:
    value = data[key]
    for val in value:
      row = str(val['subject']['category']) + "," + str(val['predicate']) + "," + str(val['object']['category'])
      writer.writerow([val['subject']['category'] , val['predicate'], val['object']['category'] ])

# Mount Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
%cd '/content/drive/MyDrive/tesi/Visual-Relationship-Detection-LTN'

# TODO creare script per scaricare dataset e piazzarlo nel posto giusto
#!unzip 'data/sg_dataset.zip' -d 'data'

!ls

/content/drive/MyDrive/tesi/Visual-Relationship-Detection-LTN
data			   refine_predictions.py
logictensornetworks.py	   relationship_phase_detection.ipynb
models			   relationship_phrase_detection.py
predicate_detection.ipynb  train.py
predicate_detection.py	   visual_relationship_dataset.py
__pycache__		   Visual-Relationship-Detection-master
README.md


# Load Data

In [3]:
from visual_relationship_dataset import *
import os
import scipy.io as sio
from PIL import Image
import copy
from refine_predictions import refine_equiv

np.set_printoptions(precision=2)
np.set_printoptions(threshold=np.inf)


img_dir = 'data/sg_dataset/sg_test_images'

# Load training data for prio statistics on the dataset
triples_of_train_data = get_data("train", False)

End of loading data


In [22]:
#idxs_of_positive_examples_of_predicates = {}

#for predicate in selected_predicates:
#    idxs_of_positive_examples_of_predicates[predicate] = np.where(np.array(predicates[triples_of_train_data[:, -2]]) == predicate)[0]

#prior_stats = np.array([len(idxs_of_positive_examples_of_predicates[pred]) for pred in selected_predicates])
#prior_freq = np.true_divide(prior_stats, np.sum(prior_stats))

sum_predicates = len(triples_of_train_data[:, -2])
predicates_annotations = triples_of_train_data[:, -2]
from collections import Counter
counter_values = Counter(predicates_annotations).values()

prior_freq = np.zeros(70)
for i, value in enumerate(counter_values):
  prior_freq[i] = Counter(predicates_annotations)[i] / sum_predicates

image_path = sio.loadmat('Visual-Relationship-Detection-master/data/imagePath.mat')
gt = sio.loadmat('Visual-Relationship-Detection-master/evaluation/gt.mat')
gt_sub_bboxes = gt['gt_sub_bboxes']
gt_obj_bboxes = gt['gt_obj_bboxes']
gt_tuple_label = gt['gt_tuple_label']

In [25]:
for img_id in range(len(gt_sub_bboxes[0])):
    if len(gt_sub_bboxes[0][img_id]) > 0:
        assert np.all(gt_sub_bboxes[0][img_id][:, 0] < gt_sub_bboxes[0][img_id][:, 2])
        assert np.all(gt_sub_bboxes[0][img_id][:, 1] < gt_sub_bboxes[0][img_id][:, 3])

        assert np.all(gt_obj_bboxes[0][img_id][:, 0] < gt_obj_bboxes[0][img_id][:, 2])
        assert np.all(gt_obj_bboxes[0][img_id][:, 1] < gt_obj_bboxes[0][img_id][:, 3])

Cambiare 2743361338_3d1a1e3b93_o.png in .jpg
Cambiare 4392556686_44d71ff5a0_o.gif in .jpg

In [26]:
features_detected_bb = []
obj_bboxes_ours = []
sub_bboxes_ours = []
obj_labels_ours = []
sub_labels_ours = []

semantic_feat_vect = np.zeros(len(types))

for pic_idx in range(gt_tuple_label.shape[1]):
    gt_sub_bboxes[0, pic_idx] = gt_sub_bboxes[0, pic_idx].astype(np.float)
    gt_obj_bboxes[0, pic_idx] = gt_obj_bboxes[0, pic_idx].astype(np.float)
    features_per_image = np.empty((0, 2*number_of_features + number_of_extra_features))
    obj_bboxes_ours_per_image = np.array([]).reshape(0, 4)
    sub_bboxes_ours_per_image = np.array([]).reshape(0, 4)
    obj_label_per_image = np.array([])
    sub_label_per_image = np.array([])

    # normalize data
    if len(gt_sub_bboxes[0, pic_idx]) > 0:
        img = Image.open(os.path.join(img_dir, image_path['imagePath'][0, pic_idx][0]).replace('png', 'jpg'))
        width, height = img.size
        normalized_gt_sub_bboxes = copy.deepcopy(gt_sub_bboxes[0, pic_idx])
        normalized_gt_sub_bboxes[:, -4] /= width
        normalized_gt_sub_bboxes[:, -3] /= height
        normalized_gt_sub_bboxes[:, -2] /= width
        normalized_gt_sub_bboxes[:, -1] /= height

        normalized_gt_obj_bboxes = copy.deepcopy(gt_obj_bboxes[0, pic_idx])
        normalized_gt_obj_bboxes[:, -4] /= width
        normalized_gt_obj_bboxes[:, -3] /= height
        normalized_gt_obj_bboxes[:, -2] /= width
        normalized_gt_obj_bboxes[:, -1] /= height

    for bb_idx in range(len(gt_tuple_label[0, pic_idx])):
        bb1 = normalized_gt_sub_bboxes[bb_idx]
        bb2 = normalized_gt_obj_bboxes[bb_idx]
        sub_label_per_image = np.append(sub_label_per_image, gt_tuple_label[0, pic_idx][bb_idx, 0])
        obj_label_per_image = np.append(obj_label_per_image, gt_tuple_label[0, pic_idx][bb_idx, 2])

        feat_vect_bb1 = np.hstack((semantic_feat_vect, bb1))
        feat_vect_bb2 = np.hstack((semantic_feat_vect, bb2))
        feat_vect_bb1[gt_tuple_label[0, pic_idx][bb_idx, 0]] = 1.0
        feat_vect_bb2[gt_tuple_label[0, pic_idx][bb_idx, 2]] = 1.0
        feat_vec_pair = np.hstack((feat_vect_bb1, feat_vect_bb2, computing_extended_features(bb1, bb2)))

        features_per_image = np.vstack((features_per_image, feat_vec_pair[np.newaxis, :]))
        sub_bboxes_ours_per_image = np.vstack((sub_bboxes_ours_per_image, gt_sub_bboxes[0, pic_idx][bb_idx]))
        obj_bboxes_ours_per_image = np.vstack((obj_bboxes_ours_per_image, gt_obj_bboxes[0, pic_idx][bb_idx]))

    features_detected_bb.append(features_per_image)
    obj_bboxes_ours.append(obj_bboxes_ours_per_image)
    sub_bboxes_ours.append(sub_bboxes_ours_per_image)
    obj_labels_ours.append(obj_label_per_image)
    sub_labels_ours.append(sub_label_per_image)

In [27]:
model_list = [
    "models/KB_wc_2500.ckpt",
    "models/KB_nc_2500.ckpt"]

config = tf.ConfigProto(device_count={'GPU': 1})

for model_type in model_list:

    model = model_type
    model_label = model.split("/")[-1][:-5]
    print(model.upper())
    obj_bboxes_ours_output = []
    sub_bboxes_ours_output = []
    predicted_predicates_values_tensor = tf.concat([isInRelation[predicate].tensor() for predicate in selected_predicates], 1)
    # concat edited, axis was in the wrong place
    saver = tf.train.Saver()
    sess = tf.Session(config=config)
    saver.restore(sess, model)
    rlp_confs_ours = []
    rlp_labels_ours = []
    error_an = np.array([]).reshape(0, 8)

    for pic_idx in range(gt_tuple_label.shape[1]):
        if pic_idx % 100 == 0:
            print("Eval img" + str(pic_idx))
        values_of_predicates = np.array([], dtype=np.float32).reshape(0, 70)
        values_of_predicates = sess.run(predicted_predicates_values_tensor, {pairs_of_objects.tensor: features_detected_bb[pic_idx]}) 
        values_of_predicates = refine_equiv(values_of_predicates, selected_predicates, "max")
        values_of_predicates = np.multiply(values_of_predicates, prior_freq)


        conf_predicates_per_image = values_of_predicates.flatten('F')
        sub_bboxes_ours_output.append(np.tile(sub_bboxes_ours[pic_idx], (len(selected_predicates), 1)))
        obj_bboxes_ours_output.append(np.tile(obj_bboxes_ours[pic_idx], (len(selected_predicates), 1)))
        # Matlab indices start from 1
        label_predicates_per_image = np.hstack((np.tile(sub_labels_ours[pic_idx], len(selected_predicates))[:, np.newaxis],
                                                np.repeat(np.array(range(1, len(selected_predicates) + 1)), len(features_detected_bb[pic_idx]))[:, np.newaxis],
                                                np.tile(obj_labels_ours[pic_idx], len(selected_predicates))[:, np.newaxis]))

        rlp_confs_ours.append(conf_predicates_per_image[:, np.newaxis])
        rlp_labels_ours.append(label_predicates_per_image)
    sess.close()

    sio.savemat("Visual-Relationship-Detection-master/results_LTN/predicate_det_result_" + model_label + ".mat",
                {'sub_bboxes_ours': sub_bboxes_ours_output,
                 'obj_bboxes_ours': obj_bboxes_ours_output,
                 'rlp_confs_ours': rlp_confs_ours,
                 'rlp_labels_ours': rlp_labels_ours})

MODELS/KB_WC_2500.CKPT
INFO:tensorflow:Restoring parameters from models/KB_wc_2500.ckpt
Eval img0
Eval img100
Eval img200
Eval img300
Eval img400
Eval img500
Eval img600
Eval img700
Eval img800
Eval img900
MODELS/KB_NC_2500.CKPT
INFO:tensorflow:Restoring parameters from models/KB_nc_2500.ckpt
Eval img0
Eval img100
Eval img200
Eval img300
Eval img400
Eval img500
Eval img600
Eval img700
Eval img800
Eval img900


In [29]:
tf.concat(1, [isInRelation[predicate].tensor() for predicate in selected_predicates])

<tf.Tensor 'concat_8/concat:0' shape=() dtype=int32>

In [28]:
tf.concat([isInRelation[predicate].tensor() for predicate in selected_predicates], 1)

<tf.Tensor 'concat_7:0' shape=<unknown> dtype=float32>